In [1]:
%matplotlib notebook

이 장의 예제는 아래의 링크에서 아이디어를 얻어서 새로 구현했습니다.

#### reference: https://towardsdatascience.com/attention-seq2seq-with-pytorch-learning-to-invert-a-sequence-34faf4133e53

In [2]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from tqdm import tqdm
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

In [3]:
torch.__version__

'1.12.0+cu113'

In [4]:
torch.__version__

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
MAX_LENGTH = 15

In [6]:
bos = 0
eos = 1
pad = 2

In [7]:
inverse_map = {
    'a':'z',
    'b':'y',
    'c':'x',
    'd':'w',
    'e':'v',
    'f':'u',
    'g':'t',
    'h':'s',
    'i':'r',
    'j':'q',
    'k':'p',
    'l':'o',
    'm':'n',
    'n':'m',
    'o':'l',
    'p':'k',
    'q':'j',
    'r':'i',
    's':'h',
    't':'g',
    'u':'f',
    'v':'e',
    'w':'d',
    'x':'c',
    'y':'b',
    'z':'a'
}

In [8]:
a2i = {
    '<s>':0,
    '</s>':1,
    '<pad>':2,
    'a':3,
    'b':4,
    'c':5,
    'd':6,
    'e':7,
    'f':8,
    'g':9,
    'h':10,
    'i':11,
    'j':12,
    'k':13,
    'l':14,
    'm':15,
    'n':16,
    'o':17,
    'p':18,
    'q':19,
    'r':20,
    's':21,
    't':22,
    'u':23,
    'v':24,
    'w':25,
    'x':26,
    'y':27,
    'z':28,
}

In [9]:
i2a = {v:k for k, v in a2i.items()}

In [10]:
i2a

{0: '<s>',
 1: '</s>',
 2: '<pad>',
 3: 'a',
 4: 'b',
 5: 'c',
 6: 'd',
 7: 'e',
 8: 'f',
 9: 'g',
 10: 'h',
 11: 'i',
 12: 'j',
 13: 'k',
 14: 'l',
 15: 'm',
 16: 'n',
 17: 'o',
 18: 'p',
 19: 'q',
 20: 'r',
 21: 's',
 22: 't',
 23: 'u',
 24: 'v',
 25: 'w',
 26: 'x',
 27: 'y',
 28: 'z'}

In [11]:
def generate_random_alphabet_index():
    random_length = np.random.randint(10, MAX_LENGTH-2)    # -2 because of <s> and </s>
    random_alphabet_index = np.random.randint(0, 26, random_length) + 3
    return random_alphabet_index.tolist(), random_length

In [12]:
def make_alphabet_tensor():
    indice, length = generate_random_alphabet_index()
    indice = indice + [pad]*(MAX_LENGTH-length)
    return torch.tensor([indice]).to(device)

In [13]:
class AlphabetToyDataset(Dataset):
    def __init__(self, n_dataset=1000):
        bos = 0
        eos = 1
        pad = 2
        self.inputs = []
        self.labels = []
        self.length = []
        for _ in range(n_dataset):
            # make input example
            aindex, alen = generate_random_alphabet_index()
            
            # index to alphabet
            #alphabet = list(map(lambda a: i2a[a], aindex))
            
            # inversing
            #inversed_alphabet = list(map(lambda a: inverse_map[a], alphabet))
            
            # alphabet to index
            #iindex = list(map(lambda ia: a2i[ia], inversed_alphabet))
            iindex = aindex[::-1]
            
            # add bos, eos and pad
            n_pad = MAX_LENGTH - len(aindex) - 1
            aindex = aindex + [eos] + [pad]*n_pad
            iindex = iindex + [eos] + [pad]*n_pad
            
            # add to examples
            self.inputs.append(aindex)
            self.labels.append(iindex)
            self.length.append(alen)
            
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        return [
            torch.tensor(self.inputs[index], dtype=torch.long),
            torch.tensor(self.labels[index], dtype=torch.long),
            torch.tensor(self.length[index], dtype=torch.long)
        ]

In [14]:
train_dataset = AlphabetToyDataset(n_dataset=3000)
valid_dataset = AlphabetToyDataset(n_dataset=300)

In [15]:
len(train_dataset), len(valid_dataset)

(3000, 300)

In [16]:
def convert_index_to_alphabet(index):
    alphabet = list(map(lambda i: i2a[i], index))
    return ' '.join(alphabet)

In [17]:
for i in range(3):
    ex = train_dataset[i]
    aindex, iindex, length = ex
    
    print('aindex_{}: {}'.format(len(aindex), convert_index_to_alphabet(aindex.numpy())))
    print('iindex_{}: {}'.format(len(iindex), convert_index_to_alphabet(iindex.numpy())))
    print('** aindex_{}: {}'.format(len(aindex), aindex))
    print('** iindex_{}: {}'.format(len(iindex), iindex))
    print('------------')

aindex_15: s t k l h b n v u r v </s> <pad> <pad> <pad>
iindex_15: v r u v n b h l k t s </s> <pad> <pad> <pad>
** aindex_15: tensor([21, 22, 13, 14, 10,  4, 16, 24, 23, 20, 24,  1,  2,  2,  2])
** iindex_15: tensor([24, 20, 23, 24, 16,  4, 10, 14, 13, 22, 21,  1,  2,  2,  2])
------------
aindex_15: n h p v d m u d y f f g </s> <pad> <pad>
iindex_15: g f f y d u m d v p h n </s> <pad> <pad>
** aindex_15: tensor([16, 10, 18, 24,  6, 15, 23,  6, 27,  8,  8,  9,  1,  2,  2])
** iindex_15: tensor([ 9,  8,  8, 27,  6, 23, 15,  6, 24, 18, 10, 16,  1,  2,  2])
------------
aindex_15: h h z w p d h r g u q f </s> <pad> <pad>
iindex_15: f q u g r h d p w z h h </s> <pad> <pad>
** aindex_15: tensor([10, 10, 28, 25, 18,  6, 10, 20,  9, 23, 19,  8,  1,  2,  2])
** iindex_15: tensor([ 8, 19, 23,  9, 20, 10,  6, 18, 25, 28, 10, 10,  1,  2,  2])
------------


In [18]:
def collate_fn(batch):
    inputs = pad_sequence([b[0] for b in batch], batch_first=True)
    targets = pad_sequence([b[1] for b in batch], batch_first=True)
    lengths = torch.stack([b[2] for b in batch])

    lengths, indice = torch.sort(lengths, descending=True)
    inputs = inputs[indice]
    targets = targets[indice]
    return inputs, targets, lengths

In [19]:
train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=16)
valid_dataloader = DataLoader(valid_dataset, collate_fn=collate_fn, batch_size=1)

In [20]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

    def forward(self, inputs, hidden):
        '''
        Input Parameters
        - inputs: (B,M)
        - hidden: (1,B,H)
        
        Output returns
        - output: (B,M,H)
        - hidden: (1,B,H)
        
        Logging outputs
        ** inputs: torch.Size([16, 5])
        ** hidden: torch.Size([1, 16, 256])
        ** embedded: torch.Size([16, 5, 256])
        ** final-output: torch.Size([16, 5, 256])
        ** final-hidden: torch.Size([1, 16, 256])
        '''
        #print('** inputs: {}'.format(inputs.shape))
        #print('** hidden: {}'.format(hidden.shape))
        embedded = self.embedding(inputs)    # (B,M,H)
        #print('** embedded: {}'.format(embedded.shape))
        output, hidden = self.gru(embedded, hidden)    # (B,M,H), (1,B,H)
        #print('** final-output: {}'.format(output.shape))
        #print('** final-hidden: {}'.format(hidden.shape))
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [21]:
hidden_size = 256
enc = EncoderRNN(26+3, hidden_size).to(device)

In [22]:
inp = torch.from_numpy(np.array([
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
]))
inp = inp.to(device)
hid = enc.initHidden(inp.size(0))

In [23]:
o, h = enc(inp, hid)
o.shape, h.shape

Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda-11.2/lib64/libcublas.so.11: undefined symbol: cublasGetSmCountTarget


(torch.Size([16, 5, 256]), torch.Size([1, 16, 256]))

In [24]:
# without Attention
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        #print('DecoderRNN_INPUT: input: {} → {:.4f}'.format(input.shape, input.sum()))
        #print('DecoderRNN_INPUT: hidden: {} → {:.4f}'.format(hidden.shape, hidden.sum()))
        
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        #print('** output: {} → {:.4f}'.format(output.shape, output.sum()))
        
        
        output, hidden = self.gru(output, hidden)
        #print('** gru-output: {} → {:.4f}'.format(output.shape, output.sum()))
        #print('** gru-hidden: {} → {:.4f}'.format(hidden.shape, hidden.sum()))
        
        output = self.softmax(self.out(output[0]))
        #print('** hidden: {}'.format(hidden.shape))
        
        #print('DecoderRNN_OUTPUT: output: {} → {:.4f}'.format(output.shape, output.sum()))
        #print('DecoderRNN_OUTPUT: hidden: {} → {:.4f}'.format(hidden.shape, hidden.sum()))
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [25]:
# with attention
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, batch_first=True)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, inputs, hidden, encoder_outputs):
        '''
        Input Parameters
        - inputs: (B,1)
        - hidden: (1,B,H)
        - encoder_outputs: (B,M,H)
        
        Output returns
        - output: (B,1,O)
        - hidden: (B,1,H)
        - attn_weights: (B,1,M)
        
        Logging outputs
        ** inputs: torch.Size([16, 1])
        ** hidden: torch.Size(v[1, 16, 256])
        ** encoder_outputs: torch.Size([16, 15, 256])
        ** embedded: torch.Size([16, 1, 256])
        ** attn_weights: torch.Size([16, 1, 15])
        ** attn_weights: tensor([[[0.1068, 0.0260, 0.0466, 0.0421, 0.0885, 0.0843, 0.0718, 0.0575,
                  0.0361, 0.0273, 0.0339, 0.1882, 0.0606, 0.0723, 0.0581]],

                [[0.1634, 0.0374, 0.0518, 0.0382, 0.0771, 0.0577, 0.0624, 0.0658,
                  0.0286, 0.0351, 0.0336, 0.1435, 0.0603, 0.0940, 0.0510]],

                [[0.1191, 0.0284, 0.0447, 0.0361, 0.0816, 0.0879, 0.0675, 0.0523,
                  0.0293, 0.0273, 0.0535, 0.1981, 0.0588, 0.0728, 0.0427]],

                [[0.1043, 0.0357, 0.0479, 0.0426, 0.1006, 0.0658, 0.0556, 0.0538,
                  0.0372, 0.0476, 0.0348, 0.1877, 0.0641, 0.0705, 0.0516]],

                [[0.1179, 0.0354, 0.0404, 0.0565, 0.0785, 0.0645, 0.0700, 0.0685,
                  0.0380, 0.0414, 0.0456, 0.1519, 0.0710, 0.0682, 0.0523]],

                [[0.1363, 0.0500, 0.0404, 0.0383, 0.0748, 0.0674, 0.0733, 0.0520,
                  0.0288, 0.0306, 0.0466, 0.1517, 0.0769, 0.0932, 0.0395]],

                [[0.1429, 0.0315, 0.0380, 0.0411, 0.0754, 0.0555, 0.0701, 0.0626,
                  0.0337, 0.0311, 0.0398, 0.1486, 0.0813, 0.0885, 0.0599]],

                [[0.1183, 0.0333, 0.0422, 0.0445, 0.0842, 0.0600, 0.0854, 0.0732,
                  0.0299, 0.0447, 0.0299, 0.1091, 0.0865, 0.0831, 0.0757]],

                [[0.1145, 0.0353, 0.0372, 0.0386, 0.0867, 0.0568, 0.0840, 0.0514,
                  0.0393, 0.0285, 0.0397, 0.1800, 0.0759, 0.0755, 0.0565]],

                [[0.1614, 0.0251, 0.0357, 0.0470, 0.0903, 0.0756, 0.0521, 0.0419,
                  0.0305, 0.0227, 0.0343, 0.1683, 0.0643, 0.0891, 0.0616]],

                [[0.1152, 0.0248, 0.0413, 0.0535, 0.0836, 0.0599, 0.0707, 0.0559,
                  0.0333, 0.0394, 0.0348, 0.1688, 0.0728, 0.0880, 0.0582]],

                [[0.1475, 0.0310, 0.0462, 0.0446, 0.0865, 0.0484, 0.0775, 0.0775,
                  0.0378, 0.0357, 0.0360, 0.1070, 0.0689, 0.1070, 0.0485]],

                [[0.1168, 0.0299, 0.0347, 0.0407, 0.0787, 0.0605, 0.0683, 0.0517,
                  0.0312, 0.0334, 0.0426, 0.1895, 0.0739, 0.0914, 0.0565]],

                [[0.1122, 0.0315, 0.0371, 0.0409, 0.1061, 0.0805, 0.0861, 0.0475,
                  0.0399, 0.0262, 0.0330, 0.1467, 0.0676, 0.0788, 0.0660]],

                [[0.1491, 0.0363, 0.0410, 0.0398, 0.0919, 0.0586, 0.0605, 0.0647,
                  0.0378, 0.0277, 0.0356, 0.1396, 0.0699, 0.0981, 0.0495]],

                [[0.1127, 0.0238, 0.0486, 0.0385, 0.0809, 0.0726, 0.0829, 0.0495,
                  0.0441, 0.0275, 0.0412, 0.1654, 0.0627, 0.0825, 0.0672]]],
               device='cuda:0', grad_fn=<SoftmaxBackward>)
        ** attn_weights.sum(): torch.Size([])
        ** attn_weights.sum(): 16.0000
        ** attn_applied: torch.Size([16, 1, 256])
        ** output: torch.Size([16, 1, 512])
        ** output: torch.Size([16, 1, 256])
        ** output: torch.Size([16, 1, 256])
        ** hidden: torch.Size([1, 16, 256])
        ** gru-output: torch.Size([16, 1, 256])
        ** gru-hidden: torch.Size([1, 16, 256])
        ** final-output: torch.Size([16, 1, 29])
        ** final-hidden: torch.Size([1, 16, 256])
        ** final-attn_weights: torch.Size([16, 1, 15])
        '''
        embedded = self.embedding(inputs)    # (B,1,H)
        embedded = self.dropout(embedded)
        
        # query: embedded
        # key: hidden
        # value: encoder_outputs
        
        attn_weights = F.softmax(
            self.attn(
                torch.cat((embedded, hidden.transpose(0, 1)), -1)    # (B,1,2H)
            ),    # (B,1,M)
            dim=-1)    # (B,1,M)
        #print('** attn_weights: {}'.format(attn_weights.shape))
        #print('** attn_weights: {}'.format(attn_weights))
        #print('** attn_weights.sum(): {}'.format(attn_weights.sum().shape))
        #print('** attn_weights.sum(): {:.4f}'.format(attn_weights.sum()))
        
        attn_applied = torch.bmm(attn_weights, encoder_outputs)    # (B,1,H)
        #print('** attn_applied: {}'.format(attn_applied.shape))
        #return

        output = torch.cat((embedded, attn_applied), -1)    # (B,1,2H)
        #print('** output: {}'.format(output.shape))
        output = self.attn_combine(output)    # (B,1,H)
        #print('** output: {}'.format(output.shape))    # (B,1,H)
        
        output = F.relu(output)
        #print('** output: {}'.format(output.shape))    # (B,1,H)
        #print('** hidden: {}'.format(hidden.shape))    # (B,1,H)
        output, hidden = self.gru(output, hidden)    # (B,1,H) (1,B,H)
        #print('** gru-output: {}'.format(output.shape))
        #print('** gru-hidden: {}'.format(hidden.shape))

        output = F.log_softmax(self.out(output), dim=-1)
        #print('** final-output: {}'.format(output.shape))
        #print('** final-hidden: {}'.format(hidden.shape))
        #print('** final-attn_weights: {}'.format(attn_weights.shape))
        return output, hidden, attn_weights

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [26]:
teacher_forcing_ratio = 0.5

In [27]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH, with_attention=True):
    batch_size = input_tensor.size(0)
    encoder_hidden = encoder.initHidden(batch_size)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(1)
    target_length = target_tensor.size(1)
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0

    encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden)
    
    decoder_input = torch.tensor([bos]*batch_size, device=device)
    decoder_input = decoder_input.unsqueeze(-1)    # (B,1)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if np.random.random() < teacher_forcing_ratio else False
    
    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input,    # (B,1)
            decoder_hidden,   # (1,B,H)
            encoder_outputs   # (B,M,H)
        )
        decoder_output = decoder_output.squeeze(1)
        
        loss += criterion(decoder_output, target_tensor[:,di])
        
        if use_teacher_forcing:
            decoder_input = target_tensor[:,di].unsqueeze(-1)    # (B,1)
        else:
            topv, topi = decoder_output.topk(1)
            decoder_input = topi    # (B,1)

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [28]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [29]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01, with_attention=True):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # print_every 마다 초기화
    plot_loss_total = 0  # plot_every 마다 초기화

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for iter, batch in enumerate(train_dataloader):
        input_tensor, target_tensor, length_tensor = batch
        input_tensor = input_tensor.to(device)
        target_tensor = target_tensor.to(device)
        length_tensor = length_tensor.to(device)
        
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion, with_attention=with_attention)
        print_loss_total += loss
        plot_loss_total += loss

        if (iter+1) % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, (iter+1) / n_iters),
                                         (iter+1), (iter+1) / n_iters * 100, print_loss_avg))

        if (iter+1) % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [30]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # 주기적인 간격에 이 locator가 tick을 설정
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [31]:
def evaluate(encoder, decoder, input_tensor, max_length=MAX_LENGTH):
    with torch.no_grad():
        batch_size = input_tensor.size(0)
        encoder_hidden = encoder.initHidden(batch_size)
        #print(input_tensor.shape, encoder_hidden.shape)
        encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden)
        
        decoder_input = torch.tensor([bos]*batch_size, device=device)
        decoder_input = decoder_input.unsqueeze(-1)    # (B,1)
        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            #print('** ', decoder_input.shape, decoder_hidden.shape, encoder_outputs.shape)
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input,    # (B,1)
                decoder_hidden,   # (1,B,H)
                encoder_outputs   # (B,M,H)
            )
            decoder_output = decoder_output.squeeze(1)
            decoder_attentions[:, di] = decoder_attention
            
            topv, topi = decoder_output.topk(1)
            decoder_input = topi    # (B,1)
            if topi.item() == eos:
                decoded_words.append('</s>')
                break
            else:
                decoded_words.append(i2a[topi.item()])

        return decoded_words, decoder_attentions[:di + 1]

In [32]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [33]:
for t in train_dataloader:
    print(t)
    break

(tensor([[16, 10, 18, 24,  6, 15, 23,  6, 27,  8,  8,  9,  1,  2,  2],
        [10, 10, 28, 25, 18,  6, 10, 20,  9, 23, 19,  8,  1,  2,  2],
        [20, 25, 27,  4, 18, 18,  8, 15, 26, 14, 28, 28,  1,  2,  2],
        [16, 15, 21, 16,  8, 17, 23,  3, 28, 12, 25, 27,  1,  2,  2],
        [24, 22,  4, 22, 15, 26,  4,  3,  8, 28, 13,  5,  1,  2,  2],
        [14, 21, 22, 21,  5,  9, 17, 10, 25,  7, 19,  5,  1,  2,  2],
        [ 6,  6, 17, 13, 17, 17, 15, 14, 17,  3, 26,  3,  1,  2,  2],
        [21, 22, 13, 14, 10,  4, 16, 24, 23, 20, 24,  1,  2,  2,  2],
        [ 5, 18,  9, 19, 22, 15, 28, 11, 28, 25,  9,  1,  2,  2,  2],
        [28, 10, 16, 16, 14, 25,  3, 11,  3,  8, 24,  1,  2,  2,  2],
        [12, 17, 14, 20, 13,  4,  5, 22, 18,  4,  1,  2,  2,  2,  2],
        [ 6, 19,  3, 23,  3,  8, 28, 22, 28, 11,  1,  2,  2,  2,  2],
        [11, 12, 10,  9, 21,  6, 21, 24, 19,  7,  1,  2,  2,  2,  2],
        [11, 18, 21,  9, 24, 19, 16,  3,  5,  5,  1,  2,  2,  2,  2],
        [ 3,  7,  5

In [34]:
hidden_size = 256
encoder1 = EncoderRNN(26+3, hidden_size).to(device)
decoder1 = AttnDecoderRNN(hidden_size, 26+3, dropout_p=0.1).to(device)

In [35]:
# hidden_size = 256
# encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
# decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

In [36]:
#### Input validation

In [37]:
# trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

In [38]:
for _ in range(15):
    trainIters(encoder1, decoder1, 75000, print_every=30, with_attention=True)

0m 0s (- 15m 58s) (30 0%) 3.1027
0m 0s (- 14m 25s) (60 0%) 2.9237
0m 1s (- 13m 53s) (90 0%) 2.7904
0m 1s (- 13m 37s) (120 0%) 2.7562
0m 1s (- 13m 27s) (150 0%) 2.7285
0m 1s (- 13m 20s) (180 0%) 2.7401
0m 0s (- 12m 49s) (30 0%) 2.6372
0m 0s (- 12m 49s) (60 0%) 2.6248
0m 0s (- 12m 48s) (90 0%) 2.6172
0m 1s (- 12m 48s) (120 0%) 2.6859
0m 1s (- 12m 48s) (150 0%) 2.5440
0m 1s (- 12m 47s) (180 0%) 2.6206
0m 0s (- 12m 53s) (30 0%) 2.5959
0m 0s (- 12m 51s) (60 0%) 2.5673
0m 0s (- 12m 50s) (90 0%) 2.5720
0m 1s (- 12m 49s) (120 0%) 2.5733
0m 1s (- 12m 48s) (150 0%) 2.4534
0m 1s (- 12m 47s) (180 0%) 2.4956
0m 0s (- 12m 49s) (30 0%) 2.4694
0m 0s (- 12m 49s) (60 0%) 2.4199
0m 0s (- 12m 49s) (90 0%) 2.4581
0m 1s (- 12m 52s) (120 0%) 2.4054
0m 1s (- 12m 51s) (150 0%) 2.3592
0m 1s (- 12m 51s) (180 0%) 2.2278
0m 0s (- 12m 52s) (30 0%) 2.1803
0m 0s (- 12m 52s) (60 0%) 1.8736
0m 0s (- 12m 51s) (90 0%) 1.7850
0m 1s (- 12m 51s) (120 0%) 1.6738
0m 1s (- 12m 51s) (150 0%) 1.4032
0m 1s (- 12m 50s) (180 0%) 1.

/tmp/ipykernel_104972/734353137.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


0m 0s (- 12m 55s) (30 0%) 0.1137
0m 0s (- 12m 54s) (60 0%) 0.1119
0m 0s (- 12m 53s) (90 0%) 0.0994
0m 1s (- 12m 54s) (120 0%) 0.1085
0m 1s (- 12m 53s) (150 0%) 0.1000
0m 1s (- 12m 53s) (180 0%) 0.0951
0m 0s (- 12m 54s) (30 0%) 0.0950
0m 0s (- 12m 53s) (60 0%) 0.0858
0m 0s (- 12m 53s) (90 0%) 0.0921
0m 1s (- 12m 53s) (120 0%) 0.0991
0m 1s (- 12m 52s) (150 0%) 0.0846
0m 1s (- 12m 52s) (180 0%) 0.0796
0m 0s (- 12m 54s) (30 0%) 0.0935
0m 0s (- 12m 53s) (60 0%) 0.0871
0m 0s (- 12m 53s) (90 0%) 0.0845
0m 1s (- 12m 53s) (120 0%) 0.0872
0m 1s (- 12m 52s) (150 0%) 0.0800
0m 1s (- 12m 52s) (180 0%) 0.0888
0m 0s (- 13m 4s) (30 0%) 0.0773
0m 0s (- 12m 59s) (60 0%) 0.0668
0m 0s (- 12m 57s) (90 0%) 0.0719
0m 1s (- 12m 56s) (120 0%) 0.0851
0m 1s (- 12m 55s) (150 0%) 0.0713
0m 1s (- 12m 54s) (180 0%) 0.0734


In [39]:
for _ in range(5):
    trainIters(encoder1, decoder1, 75000, print_every=30, with_attention=True)

0m 0s (- 12m 57s) (30 0%) 0.0722
0m 0s (- 12m 56s) (60 0%) 0.0696
0m 0s (- 12m 55s) (90 0%) 0.0643
0m 1s (- 12m 54s) (120 0%) 0.0764
0m 1s (- 12m 54s) (150 0%) 0.0660
0m 1s (- 12m 53s) (180 0%) 0.0634
0m 0s (- 12m 55s) (30 0%) 0.0661
0m 0s (- 12m 54s) (60 0%) 0.0677
0m 0s (- 12m 53s) (90 0%) 0.0658
0m 1s (- 12m 53s) (120 0%) 0.8936
0m 1s (- 12m 53s) (150 0%) 0.1331
0m 1s (- 12m 52s) (180 0%) 0.0875
0m 0s (- 12m 56s) (30 0%) 0.0686
0m 0s (- 12m 54s) (60 0%) 0.0633
0m 0s (- 12m 54s) (90 0%) 0.0702
0m 1s (- 12m 53s) (120 0%) 0.0713
0m 1s (- 12m 53s) (150 0%) 0.0558
0m 1s (- 12m 52s) (180 0%) 0.0591
0m 0s (- 12m 54s) (30 0%) 0.0610
0m 0s (- 12m 54s) (60 0%) 0.0568
0m 0s (- 12m 54s) (90 0%) 0.0504
0m 1s (- 12m 53s) (120 0%) 0.0642
0m 1s (- 12m 53s) (150 0%) 0.0457
0m 1s (- 12m 52s) (180 0%) 0.0526
0m 0s (- 12m 56s) (30 0%) 0.0642
0m 0s (- 13m 4s) (60 0%) 0.0466
0m 0s (- 13m 0s) (90 0%) 0.0510
0m 1s (- 12m 58s) (120 0%) 0.0620
0m 1s (- 12m 57s) (150 0%) 0.0410
0m 1s (- 12m 57s) (180 0%) 0.04

In [40]:
# trainIters(encoder1, decoder1, 75000, print_every=100, with_attention=True)

In [41]:
for d in valid_dataloader:
    print(d[0].shape)
    break

torch.Size([1, 15])


In [42]:
for d in valid_dataloader:
    input_tensor, target_tensor, length_tensor = d
    input_string = list(map(lambda i: i2a[i], input_tensor.numpy()[0]))
    output_string = list(map(lambda i: i2a[i], target_tensor.numpy()[0]))
    input_string = np.array(input_string)
    output_string = np.array(output_string)
    print('input: {}'.format(input_string))
    print('output: {}'.format(output_string))
    
    input_tensor = input_tensor.to(device)
    target_tensor = target_tensor.to(device)
    length_tensor = length_tensor.to(device)
    
    pred, attn_weight = evaluate(encoder1, decoder1, input_tensor)
    pred = np.array(pred)
    print('pred: {}'.format(pred))
    print('-------------')
    #break

input: ['v' 'q' 'h' 'l' 'g' 'a' 'z' 'x' 'p' 's' 'q' 'z' '</s>' '<pad>' '<pad>']
output: ['z' 'q' 's' 'p' 'x' 'z' 'a' 'g' 'l' 'h' 'q' 'v' '</s>' '<pad>' '<pad>']
pred: ['z' 'q' 'z' 'p' 'x' 'z' 'a' 'g' 'l' 'h' 'q' 'v' '</s>']
-------------
input: ['a' 'j' 's' 'f' 'c' 'q' 'a' 'j' 'p' 'n' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['n' 'p' 'j' 'a' 'q' 'c' 'f' 's' 'j' 'a' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['n' 'p' 'j' 'a' 'q' 'c' 'f' 's' 'j' 'a' '</s>']
-------------
input: ['m' 'z' 'u' 'j' 'u' 'g' 'v' 'y' 'p' 'l' 't' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['t' 'l' 'p' 'y' 'v' 'g' 'u' 'j' 'u' 'z' 'm' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['t' 'l' 'p' 'y' 'v' 'g' 'u' 'j' 'u' 'z' 'm' '</s>']
-------------
input: ['l' 'y' 'f' 'j' 'i' 'p' 'q' 'l' 'w' 'v' 'n' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['n' 'v' 'w' 'l' 'q' 'p' 'i' 'j' 'f' 'y' 'l' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['n' 'v' 'w' 'l' 'q' 'p' 'i' 'j' 'f' 'y' 'l' '</s>']
-------------
input: ['u' 'e' 'j' 'b' 't

pred: ['u' 'a' 'h' 'z' 'w' 'a' 'j' 'm' 'q' 'j' '</s>']
-------------
input: ['x' 'q' 's' 'g' 'f' 'u' 'y' 'c' 'v' 'q' 'f' 's' '</s>' '<pad>' '<pad>']
output: ['s' 'f' 'q' 'v' 'c' 'y' 'u' 'f' 'g' 's' 'q' 'x' '</s>' '<pad>' '<pad>']
pred: ['s' 'f' 's' 'v' 'c' 'y' 'u' 'f' 'g' 's' 'q' 'x' '</s>']
-------------
input: ['r' 'u' 'm' 'k' 'l' 'z' 'v' 'q' 'b' 'm' 'g' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['g' 'm' 'b' 'q' 'v' 'z' 'l' 'k' 'm' 'u' 'r' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['g' 'm' 'b' 'q' 'v' 'z' 'l' 'k' 'm' 'u' 'r' '</s>']
-------------
input: ['a' 'k' 's' 'h' 'o' 'f' 'o' 'p' 'p' 't' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['t' 'p' 'p' 'o' 'f' 'o' 'h' 's' 'k' 'a' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['t' 'p' 'p' 'o' 'f' 'o' 'h' 's' 'k' 'a' '</s>']
-------------
input: ['j' 'i' 'e' 'h' 'g' 'h' 'y' 'i' 'i' 'f' 'x' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['x' 'f' 'i' 'i' 'y' 'h' 'g' 'h' 'e' 'i' 'j' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['x' 'f' 'i' 'i' 'y' 'h'

pred: ['q' 'p' 't' 't' 'x' 'y' 's' 'v' 'q' 'c' 'u' '</s>']
-------------
input: ['w' 'j' 'o' 'v' 'k' 'k' 'y' 'n' 'v' 'q' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['q' 'v' 'n' 'y' 'k' 'k' 'v' 'o' 'j' 'w' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['q' 'v' 'n' 'y' 'k' 'k' 'v' 'o' 'j' 'w' '</s>']
-------------
input: ['n' 'g' 'j' 'q' 'z' 'p' 'h' 'w' 'q' 't' 't' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['t' 't' 'q' 'w' 'h' 'p' 'z' 'q' 'j' 'g' 'n' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['t' 't' 'q' 'w' 'h' 'p' 'z' 'q' 'j' 'g' 'n' '</s>']
-------------
input: ['p' 'e' 'u' 'k' 'j' 'f' 'p' 'q' 'p' 'z' 'f' 'l' '</s>' '<pad>' '<pad>']
output: ['l' 'f' 'z' 'p' 'q' 'p' 'f' 'j' 'k' 'u' 'e' 'p' '</s>' '<pad>' '<pad>']
pred: ['l' 'f' 'z' 'p' 'q' 'p' 'f' 'j' 'k' 'u' 'e' 'p' '</s>']
-------------
input: ['z' 'p' 'v' 'l' 'l' 'q' 'k' 'o' 'l' 'a' 'r' 'e' '</s>' '<pad>' '<pad>']
output: ['e' 'r' 'a' 'l' 'o' 'k' 'q' 'l' 'l' 'v' 'p' 'z' '</s>' '<pad>' '<pad>']
pred: ['e' 'r' 'l' 'l' 'o' 'k' 'q' '

input: ['s' 'j' 'c' 'r' 'n' 'j' 'r' 'x' 'i' 'k' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['k' 'i' 'x' 'r' 'j' 'n' 'r' 'c' 'j' 's' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['k' 'i' 'x' 'r' 'j' 'n' 'r' 'c' 'j' 's' '</s>']
-------------
input: ['i' 'x' 'q' 'y' 'q' 'w' 'n' 'a' 't' 'v' 'v' 's' '</s>' '<pad>' '<pad>']
output: ['s' 'v' 'v' 't' 'a' 'n' 'w' 'q' 'y' 'q' 'x' 'i' '</s>' '<pad>' '<pad>']
pred: ['s' 'v' 't' 't' 'a' 'n' 'w' 'q' 'y' 'q' 'x' 'i' '</s>']
-------------
input: ['c' 'j' 'f' 'c' 'j' 'r' 'k' 'c' 'k' 'w' 't' 'r' '</s>' '<pad>' '<pad>']
output: ['r' 't' 'w' 'k' 'c' 'k' 'r' 'j' 'c' 'f' 'j' 'c' '</s>' '<pad>' '<pad>']
pred: ['r' 't' 'k' 'k' 'c' 'k' 'r' 'j' 'c' 'f' 'j' 'c' '</s>']
-------------
input: ['d' 'a' 'b' 'v' 'h' 'l' 'p' 'i' 'n' 't' 'z' 'w' '</s>' '<pad>' '<pad>']
output: ['w' 'z' 't' 'n' 'i' 'p' 'l' 'h' 'v' 'b' 'a' 'd' '</s>' '<pad>' '<pad>']
pred: ['w' 'z' 'i' 'n' 'i' 'p' 'l' 'h' 'v' 'b' 'a' 'd' '</s>']
-------------
input: ['e' 'z' 'n' 'r' 'h' 'p' 'c' 'j

pred: ['k' 'o' 'y' 'q' 'n' 'v' 'k' 'u' 'd' 's' '</s>']
-------------
input: ['p' 't' 'g' 'j' 'c' 'a' 'x' 'k' 'f' 'o' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['o' 'f' 'k' 'x' 'a' 'c' 'j' 'g' 't' 'p' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['o' 'f' 'k' 'x' 'a' 'c' 'j' 'g' 't' 'p' '</s>']
-------------
input: ['u' 'm' 'd' 'i' 'e' 'o' 'e' 'f' 'k' 'v' 'b' 'f' '</s>' '<pad>' '<pad>']
output: ['f' 'b' 'v' 'k' 'f' 'e' 'o' 'e' 'i' 'd' 'm' 'u' '</s>' '<pad>' '<pad>']
pred: ['f' 'b' 'f' 'k' 'f' 'e' 'o' 'e' 'i' 'd' 'm' 'u' '</s>']
-------------
input: ['l' 'b' 'h' 'b' 'w' 'm' 'p' 't' 't' 'j' 'e' 'r' '</s>' '<pad>' '<pad>']
output: ['r' 'e' 'j' 't' 't' 'p' 'm' 'w' 'b' 'h' 'b' 'l' '</s>' '<pad>' '<pad>']
pred: ['r' 'e' 't' 't' 'p' 'm' 'w' 'b' 'h' 'b' 'l' '</s>']
-------------
input: ['k' 'k' 'n' 'e' 'q' 'w' 'n' 'g' 'a' 'z' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['z' 'a' 'g' 'n' 'w' 'q' 'e' 'n' 'k' 'k' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['z' 'a' 'g' 'n' 'w' 'q' '

In [43]:
# plt.matshow(attn_weight.numpy())

In [44]:
attn_weight.shape

torch.Size([13, 15])

In [45]:
input_tensor = make_alphabet_tensor()
output_words, attentions = evaluate(encoder1, decoder1, input_tensor)

In [46]:
def showAttention(input_sentence, output_words, attentions):
    # colorbar로 그림 설정
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # 축 설정
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['</s>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # 매 틱마다 라벨 보여주기
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [47]:
def evaluateAndShowAttention():
    input_tensor = make_alphabet_tensor()
    output_words, attentions = evaluate(
        encoder1, decoder1, input_tensor)
    
    input_sentence = input_tensor[0].tolist()
    input_sentence = list(map(lambda x: i2a[x], input_sentence))
    print('original input =', input_sentence)
    input_length = input_sentence.index('<pad>')
    input_sentence = ' '.join(input_sentence[:input_length])
    print('input  =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)

In [48]:
evaluateAndShowAttention()
#evaluateAndShowAttention("elle est trop petit .")
#evaluateAndShowAttention("je ne crains pas de mourir .")
#evaluateAndShowAttention("c est un jeune directeur plein de talent .")

original input = ['i', 's', 'a', 'k', 'x', 'z', 'e', 'z', 'u', 'l', 'v', '<pad>', '<pad>', '<pad>', '<pad>']
input  = i s a k x z e z u l v
output = v u z e z x k a s i </s>


/tmp/ipykernel_104972/1632681039.py:9: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''] + input_sentence.split(' ') +
/tmp/ipykernel_104972/1632681039.py:11: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''] + output_words)
/tmp/ipykernel_104972/1632681039.py:17: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [49]:
generate_random_alphabet_index()

([12, 23, 24, 3, 14, 18, 11, 16, 20, 13, 21, 14], 12)